In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader

In [2]:
import sys
import os

module_path = os.path.abspath("/dcai/users/chache/smrt-foundation")

if module_path not in sys.path:
    sys.path.append(module_path)
    
import copy
import torch
from tqdm import tqdm
from smrt_foundation.probe import SingleIdxProbe
from smrt_foundation.model import Smrt2Vec, SmrtEncoder


device=torch.device('cuda')


In [3]:
encoder = SmrtEncoder()
encoder.cnn.r0

25

In [4]:
from smrt_foundation.dataset import LegacyMethylDataset, compute_log_normalization_stats
import polars as pl
TRAIN_PATH = '../data/01_processed/val_sets/pacbio_standard_train.parquet'
VAL_PATH =  '../data/01_processed/val_sets/pacbio_standard_test.parquet'
KINETICS_FEATURES = ['fi', 'fp', 'ri', 'rp']

df = pl.read_parquet(TRAIN_PATH).head(1_000_000)
train_means, train_stds = compute_log_normalization_stats(df, KINETICS_FEATURES)

methyl_train_ds = LegacyMethylDataset(TRAIN_PATH, train_means, train_stds, context=32, restrict_row_groups=5)
methyl_train_dl = DataLoader(methyl_train_ds,
                             # num_workers=8,
                             batch_size=256,
                             drop_last=True,
                             persistent_workers=False,
                             # prefetch_factor=5
                            )

methyl_val_ds = LegacyMethylDataset(VAL_PATH,
                                      means=train_means,
                                      stds=train_stds,
                                      context=32,
                                      restrict_row_groups=5)
methyl_val_dl = DataLoader(methyl_val_ds,
                        batch_size=256,
                        drop_last=True,
                        persistent_workers=False,
                        prefetch_factor=None)



In [5]:


EPOCHS = 30
DEVICE = torch.device('cuda')


encoder = SmrtEncoder(max_len=32)
probe = SingleIdxProbe(encoder, freeze_encoder=False).to(DEVICE)
print(encoder.cnn.r0)
optimizer = torch.optim.AdamW([
    {'params': probe.encoder.parameters(), 'lr': 2e-4},
    {'params': probe.head.parameters(), 'lr': 2e-4}
])

criterion = torch.nn.BCEWithLogitsLoss()
loss_history = []

for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}")
    probe.train()
    running_loss = 0.0
    for i, batch in enumerate(tqdm(methyl_train_dl)):
        inputs = batch['data'].to(DEVICE)
        labels = batch['label'].to(DEVICE)

        optimizer.zero_grad()
        logits = probe(inputs)
        loss = criterion(logits, labels.unsqueeze(1).to(torch.float32))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (i + 1) % 100 == 0:
            loss_history.append(running_loss / 100)
            running_loss = 0.0

    probe.eval()
    sample_count = 0
    sample_correct = 0
    # Added torch.no_grad() for validation efficiency
    with torch.no_grad():
        for batch in tqdm(methyl_val_dl):
            inputs = batch['data'].to(DEVICE)
            labels = batch['label'].to(DEVICE)

            logits = probe(inputs)
            preds = logits > 0
            correct = labels == preds.squeeze(-1)
            sample_count += correct.shape[0]
            sample_correct += correct.sum()
    
    print(f"epoch val top1_acc: {sample_correct/sample_count}")

25
Epoch 1


100%|██████████| 5166/5166 [00:16<00:00, 304.00it/s]


epoch val top1_acc: 0.7868257761001587
Epoch 2


100%|██████████| 5166/5166 [00:17<00:00, 300.19it/s]


epoch val top1_acc: 0.7983411550521851
Epoch 3


...

epoch val top1_acc: 0.7716348171234131
Epoch 29


100%|██████████| 5166/5166 [00:17<00:00, 297.11it/s]


epoch val top1_acc: 0.7711841464042664
Epoch 30


100%|██████████| 5166/5166 [00:17<00:00, 293.58it/s]

epoch val top1_acc: 0.7702699899673462


In [6]:
probe.eval()
sample_count = 0
sample_correct = 0
with torch.no_grad():
    for batch in tqdm(methyl_train_dl):
        inputs = batch['data'].to(DEVICE)
        labels = batch['label'].to(DEVICE)

        logits = probe(inputs)
        preds = logits > 0
        correct = labels == preds.squeeze(-1)
        sample_count += correct.shape[0]
        sample_correct += correct.sum()
print(f"epoch train top1_acc: {sample_correct/sample_count}")

100%|██████████| 5129/5129 [00:16<00:00, 301.93it/s]

epoch train top1_acc: 0.8433158993721008
